In [2]:
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import  GridSearchCV
from sklearn.preprocessing import StandardScaler

In [3]:
 def calculate_snsp(Y, Y_pred):
    tp = len([i for i in range(len(Y)) if (Y[i] == 1) and (Y_pred[i] == 1)])
    fn = len([i for i in range(len(Y)) if (Y[i] == 1) and (Y_pred[i] == 0)])
    tn = len([i for i in range(len(Y)) if (Y[i] == 0) and (Y_pred[i] == 0)])
    fp = len([i for i in range(len(Y)) if (Y[i] == 0) and (Y_pred[i] == 1)])
    ####
    print("tp:%d\tfn:%d\ttn:%d\tfp:%d" % (tp, fn, tn, fp))
    sensitivity = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp+tn)/(tp+fn+tn+fp)
    print("Sn: %.2f" % sensitivity)
    print("Sp: %.2f" % specificity)
    print("Accuracy: %.2f" % accuracy)

In [86]:
os.chdir("/home/galaxy/project/logistic_regression/new/")

#### read in file
# df = pd.read_table("/home/galaxy/project/logistic_regression/new/LR_total_circRNA.txt")
df = pd.read_table("/home/galaxy/project/logistic_regression/new/LR_total.txt")
matrix_list, labelMat = [], []
for class_n in [0,1]:
    df_sub = df[df['Label'] == class_n]
    labelMat += df_sub['Label'].tolist()
    df_sub = df_sub.iloc[:, 1:]
    matrix_list.append(df_sub.fillna(df_sub.mean()))
dataMat = pd.concat(matrix_list)
X,Y = dataMat, labelMat
x_train, x_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.35, stratify=Y)
### select circRNA
selected_circ = [x for x in X.columns if "circ" in x]
selected_circ = [x for x in selected_circ if (x!="circRBM39") & (x!="circSNRNP48")]
print(len(selected_circ))
# selected_circ = ['circSTXBP3', 'circHP1BP3', 'circKIF5B', 'circDEK', 'circRBM39', 'circPARG', 'circHNRNPK', 'circSENP6', 'circCHD2']
X_train, X_test = x_train[selected_circ], x_test[selected_circ]
#### normalization
## https://datascience.stackexchange.com/questions/27615/should-we-apply-normalization-to-test-data-as-well
sc = StandardScaler()
X_train_scaled = sc.fit_transform(X_train)
X_test_scaled = sc.transform(X_test)

#### preformance evaluation
svclassifier = SVC(kernel='rbf', C=1, gamma=0.1, probability=True)
svclassifier.fit(X_train_scaled, Y_train)
# svclassifier.fit(X_train, Y_train)
#### predict test set
print("########################  test set")
y_pred = svclassifier.predict(X_test_scaled)
# y_pred = svclassifier.predict(X_test)
calculate_snsp(Y_test, y_pred)
y_score = svclassifier.predict_proba(X_test_scaled)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_test, y_score)
roc_auc = metrics.auc(fpr,tpr)
print("AUC:%.2f" % roc_auc)
################################################
#### predict training set
print("########################  training set")
Y_pred = svclassifier.predict(X_train_scaled)
# Y_pred = svclassifier.predict(X_train)
calculate_snsp(Y_train, Y_pred)
y_score = svclassifier.predict_proba(X_train_scaled)[:,1]
fpr,tpr,threshold = metrics.roc_curve(Y_train, y_score)
roc_auc = metrics.auc(fpr,tpr)
print("AUC:%.2f" % roc_auc)
#### write to file
result_dir = "/home/galaxy/project/logistic_regression/new/"
t_train, t_test = x_train, x_test
t_train['Label'], t_test['Label'] = Y_train, Y_test
t_train.to_csv(os.path.join(result_dir, "Training_set.txt"), sep="\t", index=False)
t_test.to_csv(os.path.join(result_dir, "Test_set.txt"), sep="\t", index=False)

9
########################  test set
tp:14	fn:3	tn:31	fp:4
Sn: 0.82
Sp: 0.89
Accuracy: 0.87
AUC:0.86
########################  training set
tp:28	fn:4	tn:60	fp:2
Sn: 0.88
Sp: 0.97
Accuracy: 0.94
AUC:0.97


/home/galaxy/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
